<a href="https://colab.research.google.com/github/jjcheung/ctcuc22/blob/main/JJ_Cheung_Relational_Database_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Member 1: Lorraine Neo

Member 2: Cheung Jun Jie

Design and implement a >1 table relational database project with Python and SQLite.

Your project should perform the standard CRUD operations plus at least 1 aggregation function.

Task: (self proposed eg manage a contact list)

ER-diagram: 


Table specification:

Patient (**CaseID**, Patient, Age, Gender, Nationality, Status)

ClusterPatient (*CaseID*, *ClusterID*)

Cluster (**ClusterID**, Cluster)

Legend: **Bold - Primary Key**, Italics - Foreign Key


Python implementation:

In [ ]:
# program menu
def menu():
    print('COVID 19 Database Query')
    print('1. Query database')
    print('0. Exit')


def main():
    option = '-1'
    while option != '0':
        menu()
        option = input('Please enter option: ')
        if option == '1':
            query = input('Enter SQL query: ')
            for row in cursor.execute(query):
                print(row)
        elif option == '0':
            print('Thank you for using database.')
        else:
            print('Invalid option. Please try again!')
    print('Goodbye!')

main()

In [ ]:
import sqlite3
import csv

# create database connection
connection = sqlite3.connect('covid.db')

# create cursor
cursor = connection.cursor()

# create tables
cursor.execute('CREATE TABLE IF NOT EXISTS Patient (CaseID INTEGER, Patient TEXT, Age INTEGER, Gender TEXT, Nationality TEXT, Status TEXT, PRIMARY KEY(CaseID) );')
cursor.execute('CREATE TABLE IF NOT EXISTS Cluster (ClusterID INTEGER, Cluster TEXT, PRIMARY KEY(ClusterID));')
cursor.execute('CREATE TABLE IF NOT EXISTS ClusterPatient (CaseID INTEGER, ClusterID INTEGER, FOREIGN KEY(CaseID) REFERENCES Patient(CaseID), FOREIGN KEY(ClusterID) REFERENCES Cluster(ClusterID) );')

#empty lists to store values from csv
cluster = []
clusterid = 1
patient = []
clusterpatient = []

#reading csv
with open('covid.csv', newline = '') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        patient.append([row['CaseID'], row['Patient'], row['Age'], row['Gender'], row['Nationality'], row['Status']])
        if row['ClusterName'] != '': #if cluster is identified / linked
            if ';' in row['ClusterName']: #if patient belong to multiple clusters
                clist = row['ClusterName'].split(';') #list of clusters patient belong to
            else: #if patient belongs to only 1 cluster
                clist = [row['ClusterName']]
            for c in clist: 
                cid = -1 #to search for cluster id
                for lst in cluster: 
                    if c in lst: #if existing cluster
                        cid = lst[0]
                        break
                if cid == -1: #if new cluster
                    cluster.append([clusterid, c])  #append to list of clusters, with id
                    clusterpatient.append([row['CaseID'], clusterid]) #append to list for cluster-patient table
                    clusterid += 1                  #new id for next cluster
                else:
                    clusterpatient.append([row['CaseID'], cid])
        
#using values in lists to add to tables in database       
cursor.executemany('INSERT INTO Patient (CaseID, Patient, Age, Gender, Nationality, Status) VALUES (?, ?, ?, ?, ?, ?);', tuple(patient))
cursor.executemany('INSERT INTO ClusterPatient (CaseID, ClusterID) VALUES (?, ?);', tuple(clusterpatient))
cursor.executemany('INSERT INTO Cluster (ClusterID, Cluster) VALUES (?, ?);', tuple(cluster))

# query for patients in each cluster, ordered by cluster
results1 = cursor.execute("""
    SELECT p.Patient, c.Cluster
    FROM Patient AS p 
    LEFT JOIN ClusterPatient AS cp ON p.CaseID = cp.CaseID 
    LEFT JOIN Cluster AS c ON cp.ClusterID = c.ClusterID
    WHERE c.Cluster NOT NULL
    ORDER BY c.Cluster;""")
for record in results1:
    print(record)
print()

# query for Singaporean patients
results2 = cursor.execute("SELECT * from Patient WHERE Nationality = 'Singaporean';")
#results = cursor.fetchall()
for record in results2:
    print(record)
print()

# determine the average age of patients
results3 = cursor.execute("SELECT AVG(Age) from Patient;")
for record in results3:
    print(record)
print()

#updating patient status to recovered
cursor.execute("UPDATE Patient SET Status = 'Recovered' WHERE CaseID = '507';") 
connection.commit()

#delete a case from the database
cursor.execute("DELETE FROM Patient WHERE CaseID = '506';")
connection.commit()
cursor.execute("SELECT * FROM Patient;")
results = cursor.fetchall()
for record in results:
    print(record)
print()

# make changes permanent
connection.commit()

# destroy database connection (and cursor)
connection.close()

('64 year old female Singapore Citizen', 'Boulder+ Gym')
('28 year old male Singapore Citizen', 'Boulder+ Gym')
('27 year old female Singapore Citizen', 'Boulder+ Gym')
('34 year old male Singapore Citizen', 'Boulder+ Gym')
('35 year old female Filipino national', 'Boulder+ Gym')
('30 year old male Singapore Citizen', 'Boulder+ Gym')
('30 year old female Singapore Citizen', 'Boulder+ Gym')
('24 Year Old Male Singaporean', 'Boulder+ Gym')
('30 Year Old Male German', 'Boulder+ Gym')
('29 Year Old Male Singaporean', 'Boulder+ Gym')
('34 year old male Singapore Citizen', 'CNY Family Gathering at Mei Hwan Dr')
('79 year old female Singapore Citizen', 'CNY Family Gathering at Mei Hwan Dr')
('27 year old Singapore Citizen', 'CNY Family Gathering at Mei Hwan Dr')
('25 year old male Singapore Citizen', 'CNY Family Gathering at Mei Hwan Dr')
('38 year old female Singapore Citizen', 'CNY Family Gathering at Mei Hwan Dr')
('35 year old female Singapore Citizen', 'CNY Family Gathering at Mei Hwan D

In [ ]:
import sqlite3

# create database connection
connection = sqlite3.connect(<database name>)

# create cursor
cursor = connection.cursor()

# execute SQL query
cursor.execute(<SQL query>)

results = cursor.fetchall()

for record in results:
    print(record)

# make changes permanent
connection.commit()

# destroy database connection (and cursor)
connection.close()

Sample SQL queries:
```
# create table 
cursor.execute("CREATE TABLE contacts (rid INTEGER, name TEXT, dob TEXT, email TEXT, mobile TEXT);")
 
# insert
cursor.execute("INSERT INTO contacts (rid, name, dob, email, mobile) VALUES (1, 'Lim Ah Seng', '1995-01-01', 'limahseng@hotmail.com', '12345678');")


# select
cursor.execute("SELECT * FROM contacts;")

# update
cursor.execute("UPDATE contacts SET mobile = '88888888' WHERE rid = 1;")

# delete
cursor.execute("DELETE FROM contacts WHERE rid = 2;")
```